In [82]:
using DataFrames
using Chemfiles
using MIToS.PDB
using Gadfly
using Cairo
using Compose
set_default_plot_size(40cm, 14cm)

home = "/home/german/labo/18/egfr/1cavidad"
home_modos = "/home/german/labo/18/egfr/modos"
home_rtdos = "/home/german/labo/18/egfr/rtdos_log"
aa = 277

277

In [83]:
function to3(in_v::Array{Real, 1})
    return repeat(in_v .* 3, inner = [3]) .+ repeat([-2 ; -1 ; 0], outer = [length(in_v)])
end

function to3(in_v::Array{Int64, 1})
    return repeat(in_v .* 3, inner = [3]) .+ repeat([-2 ; -1 ; 0], outer = [length(in_v)])
end

to3 (generic function with 3 methods)

In [2]:
function tognm(vtor_anm)
    vtor_gnm = Array{Float64}(convert(Int64, length(vtor_anm)/3));
    vtor_anm =  vtor_anm.^2
    for i=1:convert(Int64, length(vtor_anm)/3)
        vtor_gnm[i] = sqrt(vtor_anm[i*3-2] + vtor_anm[i*3-1] + vtor_anm[i*3])
    end
    return vtor_gnm
end

tognm (generic function with 1 method)

In [3]:
function WeightedHist(in_vec, in_bins, in_weight, density = false)
    # Safety check    
    if length(in_vec) != length(in_weight)
        println("Each element of the input vector needs one weight")
        return
    end
    
    # Prepare variables
    out_counts = Array{Float64}(length(in_bins)-1)
    
    # Get weighted histogram
    for i=1:length(in_bins)-1
        if i == 1
            # Include those that fall before the beggining of the bins
            temp_bool = (in_vec .>= in_bins[i]) .& (in_vec .< in_bins[i+1]) .| (in_vec .<= in_bins[i])
            out_counts[i] = sum(in_weight[temp_bool])
        elseif i == length(in_bins)-1
            # Include those that fall after the end of the bins
            temp_bool = (in_vec .>= in_bins[i]) .& (in_vec .< in_bins[i+1]) .| (in_vec .>= in_bins[end])
            out_counts[i] = sum(in_weight[temp_bool])
        else
            temp_bool = (in_vec .>= in_bins[i]) .& (in_vec .< in_bins[i+1])
            out_counts[i] = sum(in_weight[temp_bool])
        end
    end
    
    # Get bins middle points
    out_middle = (in_bins[1:end-1] + in_bins[2:end]) / 2
    
    # Turn counts into density
    if (density == true)
        out_counts = out_counts ./ sum(out_counts) 
    end
    return out_counts, out_middle
end

WeightedHist (generic function with 2 methods)

In [4]:
function displaceAA(mod_pdb, in_vector, multiplier)
    # Preparo variables
    pdb = copy(mod_pdb)
    struct_xyz = coordinatesmatrix(pdb)
    new_struct_xyz = copy(struct_xyz)
        aa = length(pdb)
        # Determino el nro de atomos de c/ aminoácido
        natom = Array{Int64}(aa)
        [ natom[i] = length(pdb[i]) for i = 1:aa ]
        # Adapto el vector p/ darle la misma forma q la matriz de coordenadas
        vector = Array{Float64}
        const tmp_size = size(in_vector)
        const natoms = sum(natom)

        if tmp_size == (aa*3, )
                vector = transpose(reshape(in_vector, 3, aa))
        elseif tmp_size == (aa, 3)
                vector = in_vector
        else
                error("Input vector with wrong dimensions: ", tmp_size, "  ", (aa*3, 1))
        end
        sum_mat = Array{Float64}(sum(natom),3)
        cursor = 0
        for i = 1:aa
                rango = Array{Int64}(natom[i])
        if i == 1
                        sum_mat[1:natom[i], :] = repmat(transpose(vector[i, :]),
                                natom[i], 1)
                        cursor = natom[i]
                        continue
                end
                rango = collect(cursor+1:cursor + natom[i])
                sum_mat[rango, :] = repmat(transpose(vector[i, :]), natom[i], 1)
                cursor += natom[i]
        end

   # Listo, ahora puedo mover el pdb
   new_struct_xyz  = struct_xyz + sum_mat .* multiplier
   pdb = change_coordinates(pdb, new_struct_xyz);
   return pdb
end

displaceAA (generic function with 1 method)

# Obtengo Bfators

In [6]:
# Leo nombres de structuras
tmp = readdlm(joinpath(home, "pdbs.list"))
pdbs_list = convert(Array{String, 1}, reshape(tmp, length(tmp)));
aa = 277
aa_3 = aa * 3
aa_36 = aa_3 - 6
n_pdbs = length(pdbs_list)

# Leo activas e inactivas
tmp = readdlm(joinpath(home, "am.list"))
am_list = convert(Array{String, 1}, reshape(tmp, length(tmp)));

tmp = readdlm(joinpath(home, "im.list"))
im_list = convert(Array{String, 1}, reshape(tmp, length(tmp)));

cfg_filename = joinpath(home, "egfr.cfg")

"/home/german/labo/18/egfr/cavidad/egfr.cfg"

### Leo modos y frecuencias. También leo Bfactors experimentales y los índices de los residuos nonmissing

In [43]:
proto_teo_bfactors = Array{Array{Float64, 1}}(n_pdbs)
modos = Array{Array{Float64, 2}}(n_pdbs)
freqs = Array{Array{Float64, 1}}(n_pdbs)
bf_exps = Array{Array{Float64, 1}}(n_pdbs)
nonmissings = Array{Array{Int64, 1}}(n_pdbs)
nonmissings_3 = Array{Array{Int64, 1}}(n_pdbs)

for i = 1:n_pdbs
    each = pdbs_list[i]
    modos[i] = readdlm(joinpath(home_modos, each, string("modos_", each)))
    freqs[i] = convert(Array{Float64}, readdlm(joinpath(home_modos, each, string("freq_", each)))[8:end, 1])

    bf_exps[i] = readdlm(joinpath(home_modos, each, string("exp_bf_", each)))[:, 1]
    nonmissings[i] = convert(Array{Int64}, readdlm(joinpath(home_modos, each, string("nonmissing_", each)))[:, 1])
end

### Obtengo Bfactors teóricos. Obtengo βs.

In [119]:
bf_teos = Array{Array{Float64, 1}}(n_pdbs)

for i = 1:n_pdbs
    tmp = mapslices(x -> sum(x), modos[i] .^ 2 * diagm(1 ./ freqs[i]), 2)[:, 1]
    bf_teos[i] = [ tmp[j] + tmp[j+1] + tmp[j+2] for j = 1:3:length(tmp) ]
end

β = Array{Float64, 1}(n_pdbs)

for i = 1:n_pdbs
    β[i] = sum(bf_exps[i]) / sum(bf_teos[i][nonmissings[i]])
end

### Finalmente, obtengo las amplitudes de los modos

In [141]:
amps = Array{Array{Float64, 1}}(n_pdbs)
for j = 1:n_pdbs
    amps[j] = β[j] .* 1 ./ (4 .* pi .* freqs[j])
end

In [144]:
amps[2]

825-element Array{Float64,1}:
 227.068      
 131.339      
  65.8074     
  50.4649     
  47.0872     
  36.1815     
  27.6694     
  23.6741     
  22.9818     
  21.8568     
  20.8011     
  19.4189     
  18.5506     
   ⋮          
   0.000629534
   0.000627359
   0.000623193
   0.000616446
   0.00060711 
   0.000605416
   0.000604904
   0.000604515
   0.000591666
   0.00058249 
   0.000578466
   0.000575404